# PA005: High Value Customer Identification (Insiders)

# 0.0 Imports

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3
import psycopg2

from sqlalchemy import create_engine
import scipy.stats as st
from umap.umap_ import UMAP

from plotly import express as px


from sklearn.cluster import KMeans, DBSCAN

from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

from sklearn.decomposition import PCA

from sklearn.manifold import TSNE

from sklearn.ensemble import RandomForestRegressor

from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_score, silhouette_samples

from scipy.cluster.hierarchy import  linkage,dendrogram, fcluster

from sklearn.neighbors import NearestNeighbors

import s3fs
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")

# 0.2 Load Data

In [ ]:
AWS_KEY = ''
AWS_SECRET = ''
AWS_BUCKET = ''

In [ ]:
df_raw = pd.read_csv(AWS_BUCKET,encoding = 'ISO 8859-1',
    storage_options={
        "key": AWS_KEY,
        "secret": AWS_SECRET},
)
df_raw

In [ ]:
caminho = 'C:\\repos\\portfolio_projetos\\pa005_insider_clustering'
# caminho = 's3://insiders-db-pedro/'
# df_raw = pd.read_csv(caminho + '\\data\\raw\\Ecommerce.csv', encoding = 'ISO 8859-1')

In [ ]:
df_raw = df_raw.drop(['Unnamed: 8'], axis = 1)

df_raw.head()

# 1.0 Descrição os dados

In [ ]:
df1 = df_raw.copy()

## 1.1 Rename columns

In [ ]:
cols_new = ['invoice_no','stock_code','description','quantity','invoice_date','unit_price','customer_id','country']

df1.columns = cols_new
df1.head()

## 1.2 Data dimension

In [ ]:
print("Number of rows: {} \nNumber of columns: {}".format(df1.shape[0], df1.shape[1]))

## 1.3 Data types

In [ ]:
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Replace NA

In [ ]:
df_missing = df1.loc[df1['customer_id'].isna() ]
df_not_missing = df1.loc[~df1['customer_id'].isna() ]
df_not_missing.head()

In [ ]:
print('Missing => {}  and NOT missing => {}'.format(df_missing.shape[0],df_not_missing.shape[0]))

In [ ]:
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()
missing_invoice[0:9]

In [ ]:
df_not_missing.loc[df_not_missing['invoice_no'].isin( missing_invoice )]
# nao existe invoice_no do missing presente no not missing

In [ ]:
# atribuição dos novos "customer_id", apenas para nao perder os dados dos 25%
#df_not_missing['customer_id'].max()  # 18287

In [ ]:
# create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange(19000, 19000 + len(df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on = 'invoice_no', how = 'left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )
df1 = df1.drop(['customer_id_y','customer_id_x'], axis = 1)
df1.head()

In [ ]:
df1.isna().sum()

In [ ]:
# com a mudança dos missings acima, rodar novamente as analises descritivas abaixo para verificar mudanças de comportamento

In [ ]:
# # remove
# df1 = df1.dropna(subset = ['description','customer_id'])

# print('Removed data: {:.2f}%'.format(100*(1 - (df1.shape[0]/df_raw.shape[0]))))

In [ ]:
# df1.isna().sum()

## 1.6 Change dtypes

In [ ]:
# invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')
df1.head()

In [ ]:
df1.dtypes

## 1.7 Descriptive Statistics

In [ ]:
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [ ]:
# central tendency - mean / median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std / skew / kurtosis / min / max
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

# concatenate
m = pd.concat( [d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std', 'skew','kurtosis']
m

In [ ]:
# quantidade com valor negativo ? (Devolução)
# preço unitario de zero ? Promoção ?


### 1.7.2 Categorical Attributes

In [ ]:
# cat_attributes['invoice_no'].astype('int64')
# invoice_no
df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x: bool(re.search('[^0-9]+', x)) )]
print('Total of invoices with letter: ', df_letter_invoices.shape[0])
print('Total of invoices with letter and quantity < 0: ',df_letter_invoices.loc[
                                        df_letter_invoices['quantity'] < 0].shape[0])

In [ ]:
# stock code
# pegar somente letras
df1.loc[df1['stock_code'].apply( lambda x: bool(re.search('^[a-zA-Z]+$', x)) ),
                   'stock_code'].unique()

In [ ]:
# acao:  remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']


In [ ]:
# description
df1.head()

# delete description

In [ ]:
# country
df1['country'].unique()

In [ ]:
df1['country'].value_counts( normalize = True).head()

In [ ]:
df1[['country','customer_id']].drop_duplicates().groupby('country').count().sort_values(by = 'customer_id',
                                                                                        ascending = False).head()

# 2.0 Filtragem das variáveis

In [ ]:
df2 = df1.copy()

In [ ]:
df2.sort_values(['customer_id','invoice_no','description']).head(10) 
# há devolução se olhar o primeiro individuo, pois o "C" parece ser a flag para devolução

In [ ]:
# quantity (negative)
df2.loc[df2['quantity'] < 0 ].sort_values('invoice_no').head()

In [ ]:
#====== NUMERICAL ===========
# unity price > 0
df2 = df2.loc[df2['unit_price'] > 0.040]

# stock code != ['POST, D, M, DOT, CRUK]
df2 = df2.loc[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'] ) ]

# description
df2 = df2.drop(['description'], axis = 1)

# country change
df2 = df2.loc[ ~df2['country'].isin( ['Unspecified','European Community'])]


# esse cliente apareceu com compras e estornos altos após o pandas profiling na etapa de verificar outliers
df2 = df2.loc[~df2['customer_id'].isin( [16446])]


# quantity
df2_returns = df2.loc[ df2['quantity'] < 0]
df2_purchases = df2.loc[ df2['quantity'] >= 0]

# 3.0 Feature Engineering

In [ ]:
df3 = df2.copy()

## 3.1 Feature creation

In [ ]:
# data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date',
       'unit_price', 'country'], axis = 1).drop_duplicates().reset_index(drop = True)

df_ref.head()

### 3.1.1 Gross revenue

In [ ]:
# gross revenue = quantity * price (faturamento)
df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']

In [ ]:
# monetary
df_monetary = df2_purchases[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()

In [ ]:
df_ref = pd.merge(df_ref, df_monetary, on = 'customer_id',how = 'left')

df_ref.isna().sum() # tem NA por conta de ser somente as compras 'positivas'

### 3.1.2 Recency - day from last purchase

In [ ]:
# recency - last day purchase
df_recency = df2_purchases[['customer_id','invoice_date']].groupby('customer_id').max().reset_index()

# pego o df1 com max() pois eu considero que seria eu olhando pra "hoje" no meu dataset e tirando 'hoje - datas'
df_recency['recency_days'] = ( df2_purchases['invoice_date'].max() - df_recency['invoice_date']  ).dt.days
df_recency = df_recency[['customer_id','recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on = 'customer_id', how = 'left')
df_ref.isna().sum()

### 3.1.5 Quantity of unique products purchased

In [ ]:
# number of products
df_frequency = df2_purchases[['customer_id','stock_code']].groupby(
                                'customer_id').count().reset_index().rename(columns={'stock_code':'qtde_products'})
df_ref = pd.merge(df_ref, df_frequency, on = 'customer_id', how = 'left')
df_ref.isna().sum()

In [ ]:
df_ref.head()

### 3.1.7 Frequency purchase

In [ ]:
df2[['invoice_no','customer_id','invoice_date']].drop_duplicates().head()

In [ ]:
df_aux = df2_purchases[['invoice_no','customer_id','invoice_date']].drop_duplicates().groupby('customer_id').agg(
                    max_ = ('invoice_date','max'),
                    min_ = ('invoice_date','min'),
                    days_ = ('invoice_date',lambda x: (x.max() - x.min()).days + 1),
                    buy_ = ('invoice_date','count')).reset_index()

# frequency
df_aux['frequency'] = df_aux[['days_','buy_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0
                                                else 0, axis = 1)
# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id','frequency']], on = 'customer_id', how = 'left')
df_ref

In [ ]:
df_ref.isna().sum()

### 3.1.8 Number of returns

In [ ]:
df_returns = df2_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index()
df_returns.rename(columns = {'quantity':'qtde_returns'}, inplace = True)

df_returns['qtde_returns'] = df_returns['qtde_returns'] * ( -1 )

df_ref = pd.merge(df_ref, df_returns, on = 'customer_id', how = 'left')

df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

# 4.0 EDA (exploratory data analysis)

In [ ]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

In [ ]:
df4.shape

## 4.3 Space Study

In [ ]:
# original dataset
# df43 = df4.drop(['customer_id'], axis = 1).copy()

# selected dataset
cols_selected = ['customer_id','gross_revenue','recency_days','qtde_products','frequency','qtde_returns']
df43 = df4[cols_selected].drop(['customer_id'], axis = 1).copy()
df43.head()

In [ ]:
# rescaling/standartization
mm = MinMaxScaler()
# ss = StandardScaler()
# rs = RobustScaler()

df43['gross_revenue'] = mm.fit_transform( df43[['gross_revenue']] )
df43['recency_days'] = mm.fit_transform( df43[['recency_days']] )
df43['qtde_products'] = mm.fit_transform( df43[['qtde_products']] )
df43['frequency'] = mm.fit_transform( df43[['frequency']] )

df43['qtde_returns'] = mm.fit_transform( df43[['qtde_returns']] )

In [ ]:
X = df43.copy()
X.shape

### 4.3.1 PCA

In [ ]:
# pca = PCA( n_components = X.shape[1] )

# principal_components = pca.fit_transform( X )

# # plot explainable variables
# features = range( pca.n_components_ )

# plt.bar( features, pca.explained_variance_ratio_, color = 'black')

# # pca component
# df_pca = pd.DataFrame( principal_components )

In [ ]:
# sns.scatterplot( x=0, y = 1, data = df_pca );

### 4.3.2 UMAP

In [ ]:
# reducer = UMAP( random_state = 42 )
# embedding = reducer.fit_transform( X )

# # embedding
# df_umap = pd.DataFrame()
# df_umap['embedding_x'] = embedding[:,0]
# df_umap['embedding_y'] = embedding[:,1]

# # plot UMAP
# sns.scatterplot( x = 'embedding_x', y = 'embedding_y',data = df_umap)

### 4.3.3 t-SNE

In [ ]:
# reducer = TSNE( n_components = 2, random_state = 42 )
# embedding = reducer.fit_transform( X )

# # embedding
# df_tsne = pd.DataFrame()
# df_tsne['embedding_x'] = embedding[:,0]
# df_tsne['embedding_y'] = embedding[:,1]

# # plot UMAP
# sns.scatterplot( x = 'embedding_x', y = 'embedding_y',data = df_tsne)

### 4.3.4 Tree-Based Embedding

In [ ]:
# training dataset
X = df43.drop(['gross_revenue'], axis = 1)
Y = df43['gross_revenue']

# model definition
rf_model = RandomForestRegressor( n_estimators=100, random_state = 42)

# model training
rf_model.fit(X, Y)

# leaf
df_leaf = pd.DataFrame( rf_model.apply(X) )
print(df_leaf.shape)

In [ ]:
reducer = UMAP( random_state = 42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

# plot UMAP
sns.scatterplot( x = 'embedding_x', y = 'embedding_y',data = df_tree)

# 5.0 Data Preparation

In [ ]:
# tree-based embedding
df5 = df_tree.copy()
d5.to_csv(caminho + '\\src\\data\\tree_based_embedding.csv', index = False)

# tree-based embedding
# df5 = df_umap.copy()

# tree-based embedding
# df5 = df_tsne.copy()

# 7.0 Hyperparameter Fine Tunning

In [ ]:
X = df5.copy()
X.head()

In [ ]:
# clusters = [2,3,4,5,6,7]
clusters = np.arange(2, 25, 1)

## 7.1 K-Means

In [ ]:
# kmeans_list = []

# for k in clusters:
#     # model definition
#     kmeans_model = KMeans(n_clusters = k, n_init = 100, random_state = 42)

#     # model training
#     kmeans_model.fit( X )


#     # model prediction
#     labels = kmeans_model.predict( X )

#     # model evaluation
#     sil = silhouette_score( X, labels, metric = 'euclidean')
#     kmeans_list.append( sil )
# #     print(sil)

In [ ]:
# plt.plot( clusters, kmeans_list, linestyle = '--', marker = 'o', color = 'b');
# plt.xlabel('K clusters')
# plt.ylabel('Silhouette Score')
# plt.title('Silhouette Score  x  K clusters')

## 7.2 GMM

In [ ]:
# gmm_list = []

# for k in clusters:
#     # model definition
#     gmm_model = GaussianMixture(n_components = k, n_init = 10, random_state = 42)

#     # model training
#     gmm_model.fit( X )


#     # model prediction
#     labels = gmm_model.predict( X )

#     # model evaluation
#     sil = silhouette_score( X, labels, metric = 'euclidean')
#     gmm_list.append( sil )
# #     print(sil)

In [ ]:
# plt.plot( clusters, gmm_list, linestyle = '--', marker = 'o', color = 'b');
# plt.xlabel('GMM clusters')
# plt.ylabel('Silhouette Score')
# plt.title('Silhouette Score  x  GMM clusters')

## 7.3 Hierarchical Clustering

In [ ]:
# # model definition and training
# hc_model = linkage( X, 'ward')

### 7.3.1 H-Clustering Silhouette Score

In [ ]:
# hc_list = []

# for k in clusters:
#     # model definition and training
#     hc_model = linkage( X, 'ward')

#     # model predict
#     labels = fcluster( hc_model, t=k, criterion = 'maxclust' )

#     # metrics
#     sil = silhouette_score( X, labels, metric = 'euclidean' )
# #     print(sil)
#     hc_list.append(sil)
    
# plt.plot( clusters, hc_list, linestyle = '--', marker = 'o', color = 'b');

## 7.5 Results

In [ ]:
# # tree-based
# df_results = pd.DataFrame(
#     {'KMeans': kmeans_list,
#     'GMM':gmm_list,
#     'HC':hc_list} ).T

# df_results.columns = clusters
# df_results.style.highlight_max(color = 'lightgreen', axis = 1)

In [ ]:
# # umap
# df_results = pd.DataFrame(
#     {'KMeans': kmeans_list,
#     'GMM':gmm_list,
#     'HC':hc_list} ).T

# df_results.columns = clusters
# df_results.style.highlight_max(color = 'lightgreen', axis = 1)

In [ ]:
# # t-sne
# df_results = pd.DataFrame(
#     {'KMeans': kmeans_list,
#     'GMM':gmm_list,
#     'HC':hc_list} ).T

# df_results.columns = clusters
# df_results.style.highlight_max(color = 'lightgreen', axis = 1)

# 8.0 Model Training

## 8.2 GMM

In [ ]:
k = 8

gmm_model = GaussianMixture(n_components = k, n_init = 300, random_state = 32)

# model training
gmm_model.fit( X )


# model prediction
labels = gmm_model.predict( X )

# model evaluation
sil = silhouette_score( X, labels, metric = 'euclidean')
#     print(sil)

### 8.2.1 Cluster Validation

In [ ]:
# WSS
# print('WSS score: {}'.format(kmeans.inertia_))

## SS
print('SS score: {}'.format(sil))

# 9.0 Cluster Analysis

In [ ]:
df9 = X.copy()
df9['cluster'] = labels

## 9.2  Visualization Inspection - GMM

In [ ]:
sns.scatterplot(x = 'embedding_x',y='embedding_y', data = df9, hue = 'cluster', palette = 'deep') #8

## 9.2 Cluster Profile

In [ ]:
df92 = df4[cols_selected].copy()
# df92['cluster'] = labels_kmeans
df92['cluster'] = labels
df92.head()

In [ ]:
# number of customer
df_cluster = df92[['customer_id','cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_cluster'] = 100 * df_cluster['customer_id'] / df_cluster['customer_id'].sum()

# average gross revenue
df_cluster_gross_revenue = df92[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = df_cluster.merge(df_cluster_gross_revenue, on = 'cluster', how = 'inner')

# average recency days
df_cluster_recency_days = df92[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = df_cluster.merge(df_cluster_recency_days, on = 'cluster', how = 'inner')

# average qtd products
df_cluster_invoice_no = df92[['qtde_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = df_cluster.merge(df_cluster_invoice_no, on = 'cluster', how = 'inner')

# average frequency
df_cluster_invoice_no = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = df_cluster.merge(df_cluster_invoice_no, on = 'cluster', how = 'inner')

# average returns
df_cluster_invoice_no = df92[['qtde_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = df_cluster.merge(df_cluster_invoice_no, on = 'cluster', how = 'inner')


df_cluster.sort_values(by=['gross_revenue'], ascending = False)

In [ ]:
# 6 Cluster Insiders
# 3 Cluster More Products
# 2 Cluster Spend Money
# 7 Cluster Even More Products
# 0 Cluster Less Days
# 1 Cluster Less 1k
# 4 Cluster Stop Returners
# 5 Cluster More Buy

- Cluster 01 (Insiders):
    - Número de clientes: 468 (15 % do total)
    - Recência média de 21 dias
    - Compras em média de 424 compras
    - Receita média de $ 8835,90
    

- Cluster 02:
    - Número de clientes: 31 (0,71 % do total)
    - Recência média de 13 dias
    - Compras em média de 53 compras
    - Receita média de $ 40.543,52


- Cluster 03:
    - Número de clientes: 4335 (99,15 % do total)
    - Recência média de 92 dias
    - Compras em média de 4 compras
    - Receita média de $ 1372,58

# 10 EDA - Hypothesis

In [ ]:
df10 = df92.copy()
df10.head()

# 11.0 Deploy to Production

In [ ]:
for i in ['recency_days','qtde_returns','qtde_products']:
    df92[i] = df92[i].astype('int64')

In [ ]:
endpoint = 'sqlite:///' + caminho + '\\notebooks\\insiders_db.sqlite'
endpoint

In [ ]:
host = ''
username = ''
database = ''
pwd = ''
port = '5432'

endpoint = 'postgresql://{}:{}@{}/{}'.format(username, pwd, host, database)
endpoint

In [ ]:
conn = create_engine(endpoint)

In [ ]:
# query_truncate_table_insiders = """ DROP TABLE insiders"""
# conn.execute( query_truncate_table_insiders )

In [ ]:
# # create table
# query_create_insiders = """
#     CREATE TABLE insiders (
#     customer_id      INTEGER,
#     gross_revenue    REAL,
#     recency_days     INTEGER,
#     qtde_products    INTEGER,
#     frequency        REAL,
#     qtde_returns     INTEGER,
#     cluster          INTEGER
#     )
#     """

# # conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute( query_create_insiders )
# # conn.commit()
# # conn.close()

In [ ]:
# # insert data
df92.to_sql('insiders',con = conn, if_exists = 'append', index = False)

In [ ]:
# select data
query = """ SELECT * FROM insiders"""
df_query = pd.read_sql( query, conn)
df_query

In [ ]:
conn.close()

In [ ]:
# call C:\Users\pedro\miniconda3\condabin\activate.bat
# conda activate pa005insiderclustering
# cd C:\repos\portfolio_projetos\pa005_insider_clustering
# set data_python=%date:~6,4%-%date:~3,2%-%date:~0,2%%time:~0,2%:%time:~3,2%:%time:~6,2%
# set path_pa005=C:\repos\portfolio_projetos\pa005_insider_clustering
# papermill %path_pa005%\notebooks\c0.10-Deploy.ipynb %path_pa005%\notebooks\c0.11-Deploy_%data_python%.ipynb